# 指针

什么是指针?这个在python中可没有.

上语文课的时候,老师最长说的话是不是"请将课本翻到xxx页"?在那一页上有课上的内容,我们要看的是内容,但却使用页码标记内容,这就是指针的核心思想了.

**指针**是变量的地址,与之对应的是内存空间的标记,一本有页码的笔记本,在某一页上可以写着日记,也可以画着画,它只是被标记的空间而已,内容取决于需要.

### 变量的三要素

一个变量必然有三样内容:

+ 地址,内存空间的标记
+ 值,存放的具体内容
+ 变量名,在程序中用于标记变量的符号

在多数语言中还要加上一个类型,用以让编译器或者解释器知道该如何处理这个变量

### 取地址符&和指针运算符\*

+ 要取到变量的地址,只用在变量前使用`&`就可以得到了
+ 如果我们想通过变量的地址获取它的值,那只要在地址前使用\*就可以了

## 指针运算

指针也就是地址,在C++中表现为可以运算的正整数数值,因此,它和int,long之类一样,是可以运算的,所用的运算符也是通常的运算符.

## 指针与数组

数组变量实际上就是数组第一个元素的指针,因此要取第一个元素的值也可以使用指针运算符,其后面的元素也只要在数组变量的基础上加上长度后再做指针运算即可,不过最安全最直观的方式还是用下标

## 函数与指针

C++语言中的精髓之一就在于函数与指针互动了

### 指针作为参数

指针作为参数就可以理解为所谓的传引用,以地址作为参数,函数的操作会影响引用的内容,这样几百年函数的内存被回收,操作的结果也会生效

最典型的,交换两个变量的值

In [1]:
%%writefile ../src/C1/section5/exchange.cpp
#include <stdio.h>

void exchange(int* ptr_a,int* ptr_b){
    int temp;
    temp = *ptr_a;
    *ptr_a = *ptr_b;
    *ptr_b = temp;
    return;
}

int main(void) {
    int a = 78,b = 28;
    printf("before exchange a=%d,b=%d\n",a,b);
    exchange(&a,&b);
    printf("after exchange a=%d,b=%d\n",a,b);
    return 0;
}


Writing ../src/C1/section5/exchange.cpp


In [2]:
!g++-7 -o ../bin/exchange ../src/C1/section5/exchange.cpp

In [3]:
!../bin/exchange

before exchange a=78,b=28
after exchange a=28,b=78


#### 指针参数的保护

我们如果传递的是指针,那么就意味着内容是可变的,于是就又被篡改的危险,但如果我们希望不被篡改,那么最好的办法就是将这个指针引用的内容在函数中保护起来,这就用到了`const`参数了,这个被称作常量指针

In [4]:
%%writefile ../src/C1/section5/safe_ptr.cpp
#include <stdio.h>

int main(void) {
    const int a = 78,b = 28;
    int c = 18;
    const int* p = &a;
    printf("p:%d\n",*p);
    //*p = 58//报错,*p无法重新复制
    p = &b;//可以给p重新复制,
    printf("p:%d\n",*p);
    return 0;
}

Writing ../src/C1/section5/safe_ptr.cpp


In [5]:
!g++-7 -o ../bin/safe_ptr ../src/C1/section5/safe_ptr.cpp

In [6]:
!../bin/safe_ptr

p:78
p:28



### 指针作为返回值(指针函数)

如果要一个函数返回指针,那么就像下面这么定义

```c
int *f(int，int);
```
指针作为返回值就意味着可以返回复杂的数据结构以及数组了

### 函数指针

函数指针是指指向函数的指针,事实上函数一样也是一种储存在内存中的数据,如果要做一定的抽象,函数指针是必不可少的.定义一个函数指针如下:

```c
int (*f)(int,int);
```

形式上 指针函数和函数指针很像,也就是一对括号的不同而已,但差异非常大,指针函数是定义的函数,而函数指针是定义的指针


### 函数常量指针

C++中事实上并没有所谓常量函数,但我们确实可以指定指向函数的指针,或者函数的返回值指针,各种各样的指针为常数


```c
int (*const func)(int) = some_func;//指向函数的常量指针,通常会接一个初始化

void const *(*func)(int);//指定返回值为常量指针

void const *(*const func)(int) = func;//指向函数的常量指针,并指定返回值为常量指针
```

## 模拟高阶函数

对于函数式编程而言,高阶函数是一切的基础,学过python的一定不会陌生,C++中如何实现高阶函数呢

### 函数作为参数

求a到b间整数的和

```c
int sum_int(int a,int b){
    return a>b ?0:a + sum_int((a+1),b);
}
```

求a到b间的整数的立方和:

```c
int cube(int x){
    return (int)pow(x,3);
}

int sum_func(int a,int b,int (*func)(int)){

    return a>b ?0:(*func)(a) + sum_func((a+1),b,func);
}
```
求:
$$ \frac {1}{1 \cdot 3} + \frac {1}{5 \cdot 7} + \frac {1}{9 \cdot 11} +\cdots$$

它会非常缓慢地收敛到$$\frac {\pi}{8}$$

```c
float sum_func_stp(int a,int b,float (*const func)(int) , int (*const step)(int)){
    return a>b ?0:(*func)(a) + sum_func_stp((*step)(a),b,func,step);
}
float func(int x){
    return 1.0/(x*(x+2));
}
int stp(int x){
    return x+4;
}
float sum_pi(int a,int b){
    float (*const ptr_func)(int) = func;
    int (*const ptr_stp)(int) = stp;
    return sum_func_stp(a,b,ptr_func,ptr_stp);
}
```

完整的高阶函数例子:

In [7]:
%%writefile ../src/C1/section5/high_order_func.cpp

#include <stdio.h>
#include <math.h>

int sum_int(int a,int b){
    return a>b ?0:a + sum_int((a+1),b);
}

int cube(int x){
    return (int)pow(x,3);
}
int sum_func(int a,int b,int (*const func)(int)){
    return a>b ?0:(*func)(a) + sum_func((a+1),b,func);
}

int sum_cube(int a,int b){
    int (*const ptr_func)(int) = cube;
    return sum_func(a,b,ptr_func);
}
float sum_func_stp(int a,int b,float (*const func)(int) , int (*const step)(int)){
    return a>b ?0:(*func)(a) + sum_func_stp((*step)(a),b,func,step);
}
float func(int x){
    return 1.0/(x*(x+2));
}
int stp(int x){
    return x+4;
}
float sum_pi(int a,int b){
    float (*const ptr_func)(int) = func;
    int (*const ptr_stp)(int) = stp;
    return sum_func_stp(a,b,ptr_func,ptr_stp);
}


int main(void) {
    int top = 5,bottom = 1;
    printf("range(%d,%d) = %d\n", bottom,top,sum_int(bottom,top));
    printf("range(%d,%d)的立方和为%d\n", bottom,top,sum_cube(bottom,top));
    printf("sum_pi(%d,%d)为%f\n", bottom,top,sum_pi(bottom,top));
}

Writing ../src/C1/section5/high_order_func.cpp


In [8]:
!g++-7 -o ../bin/high_order_func ../src/C1/section5/high_order_func.cpp

In [9]:
!../bin/high_order_func

range(1,5) = 15
range(1,5)的立方和为225
sum_pi(1,5)为0.361905


## 空指针

C++中可以用`NULL`为任意指针变量赋值,他的意义是指针为空,或者说,指针变量没有指向任何地址

## 智能指针[C++11]

智能指针是基于RAII机制实现的类(模板)，具有指针的行为(重载了`operator*`与`operator->`操作符)，可以“智能”地销毁其所指对象。C++11中有`unique_ptr`、`shared_ptr`与`weak_ptr`等智能指针，可以对动态资源进行管理.他们在C++标准库`memory`中

### `unique_ptr`概念

`unique_ptr`'唯一'拥有其所指对象,同一时刻只能有一个`unique_ptr`指向给定对象(通过禁止拷贝语义,只有移动语义来实现).

`unique_ptr`指针本身的生命周期:从`unique_ptr`指针创建时开始,直到离开作用域.离开作用域时,若其指向对象,则将其所指对象销毁(默认使用`delete`操作符,用户可指定其他操作).

`unique_ptr`指针与其所指对象的关系:在智能指针生命周期内,可以改变智能指针所指对象,如创建智能指针时通过构造函数指定,通过`reset`方法重新指定、通过`release`方法释放所有权、通过移动语义转移所有权.

unique_ptr的基本操作例子:

In [10]:
%%writefile ../src/C1/section5/unique_ptr_test.cpp
#include <iostream>
#include <string>
#include <memory>
using std::cout;
using std::endl;
using std::string;
using std::unique_ptr;
int main(){
    //智能指针的创建 
    unique_ptr<int> u_i; //创建空智能指针
    u_i.reset(new int(3)); //绑定动态对象  
    cout<< *u_i << endl;
    unique_ptr<int> u_i2(new int(4));//创建时指定动态对象  
    cout<< *u_i2 << endl;
    //所有权的变化  
    int *p_i = u_i2.release(); //释放所有权  
    
    unique_ptr<string> u_s(new string("abc"));  
    cout<< *u_s << endl;
    unique_ptr<string> u_s2 = std::move(u_s); //所有权转移(通过移动语义)，u_s所有权转移后，变成“空指针”  
    cout<< *u_s2 << endl;
    u_s2=nullptr;//显式销毁所指对象，同时智能指针变为空指针。与u_s2.reset()等价  
}


Writing ../src/C1/section5/unique_ptr_test.cpp


In [11]:
!g++-7 -o ../bin/unique_ptr_test ../src/C1/section5/unique_ptr_test.cpp

In [12]:
!../bin/unique_ptr_test

3
4
abc
abc


unique_ptr的使用场景:

+ 动态资源的异常安全保证(利用其RAII特性)

```C++
void foo()  {
    //不安全的代码  
    X *px = new X;  
    // do something, exception may occurs  
    delete px; // may not go here  
}  
```

```C++
void foo()  {
    //异常安全的代码。无论是否异常发生，只要px指针成功创建，其析构函数都会被调用，确保动态资源被释放  
    unique_ptr<X> px(new X);  
    // do something,  
}  
```

+ 返回函数内创建的动态资源

```C++
unique_ptr<X> foo() {  
    unique_ptr<X> px(new X);  
    // do something  
    return px; //移动语义  
}
```

+ 可放在容器中

```C++
vector<unique_ptr<string>> vs { new string{“Doug”}, new string{“Adams”}};
```
    
+ 管理动态数组，因为`unique_ptr`有`unique_ptr<X[]>`重载版本,销毁动态对象时调用`delete[]`

```C++
unique_ptr<int[]> p (new int[3]{1,2,3});  
p[0] = 0;// 重载了operator[]  
```

+ `unique_ptr`默认的资源删除操作是`delete/delete[]`，若需要，可以进行自定义：

```C++
void end_connection(connection *p) { disconnect(*p); } //资源清理函数  
unique_ptr<connection, decltype(end_connection)*> //资源清理器的'类型'
        p(&c, end_connection);// 传入函数名，会自动转换为函数指针  
```

### shared_ptr概念

shared_ptr主要的功能是,管理动态创建的对象的销毁.它的基本原理就是记录对象被引用的次数,当引用次数为 0 的时候,也就是最后一个指向某对象的共享指针析构的时候,共享指针的析构函数就把指向的内存区域释放掉.
共享指针对象重载了 `operator*` 和 `operator->` , 所以你可以像通常的指针一样使用它.

他的接口和`unique_ptr`类似,但是多了几个接口:

+ `Swap()`函数：交换函数

```c++
int* a=new int(2);  
shared_ptr<int> sp(a);  
shared_ptr<int> sp1(a);  
sp.swap(sp1);  
```
就是两个shared_ptr中的px和pn都互换一下。

+ `Get()`函数：返回px

+ `Use_count()`函数：返回`*pn`，就是对象的拥有者的数量。

+ Unique函数：令`*pn=1`;让对象的拥有者的数量变为1.返回bool



有关运行效率的问题在这里就不讨论了.其它方面,`shared_ptr` 的构造要求比较高,如果对象在创建的时候没有使用共享指针存储的话,之后也不能用共享指针管理这个对象了.如果有引用循环(reference cycle), 也就是对象a有指向对象b的共享指针,对象b也有指向对象a的共享指针,那么它们都不会被析构.

尤其，在类的成员函数的编写的时候，有时我们经常希望得到"自己"的共享指针，但是这往往是无法得到的.此时也不能够从 this 指针构造自己的共享指针.

### weak_ptr概念

`weak_ptr`是为了配合`shared_ptr`而引入的一种智能指针,它更像是`shared_ptr`的一个助手而不是智能指针,因为它不具有普通指针的行为,没有重载`operator*`和`->`,它的最大作用在于协助`shared_ptr`工作，像旁观者那样观测资源的使用情况.

用法:
`weak_ptr`被设计为与`shared_ptr`共同工作，可以从一个`shared_ptr`或者另一个`weak_ptr`对象构造,获得资源的观测权.但`weak_ptr`没有共享资源,它的构造不会引起指针引用计数的增加.

使用`weak_ptr`的成员函数`use_count()`可以观测资源的引用计数,另一个成员函数`expired()`的功能等价于`use_count()==0`,但更快,表示被观测的资源(也就是`shared_ptr`的管理的资源)已经不复存在.

`weak_ptr`可以使用一个非常重要的成员函数`lock()`从被观测的`shared_ptr`获得一个可用的`shared_ptr`对象,从而操作资源.但当`expired()==true`的时候,`lock()`函数将返回一个存储空指针的`shared_ptr`.

## nullptr[C++11]

nullptr是为了解决原来C++中NULL的二义性问题而引进的一种新的类型,因为NULL实际上代表的是0.

In [13]:
%%writefile ../src/C1/section5/nullptr_test.cpp
#include <iostream>
using std::cout;
using std::endl;
        
void F(int a){  
    cout<<a<<endl;  
}  
 
void F(int *p){  
    assert(p != NULL);  
 
    cout<< p <<endl;  
}  
 
int main(){  
 
    int *p = nullptr;  
    int *q = NULL;  
    bool equal = ( p == q ); // equal的值为true，说明p和q都是空指针  
    int a = nullptr; // 编译失败，nullptr不能转型为int  
    F(0); // 在C++98中编译失败，有二义性；在C++11中调用F（int）  
    F(nullptr);  
 
    return 0;  
} 

Writing ../src/C1/section5/nullptr_test.cpp


## void指针

void指针很特殊,它有这样的性质:
+ void指针可以指向任何数据类型,如果要将void指针赋给其他类型指针，则需要强制类型转换如：`pint= (int *)pvoid`
+ **可以用任何类型的指针对void指针进行赋值和初始化**
+ void指针无法进行指针操作
    `void* p`就是一个典型的void指针,一般void指针用在内存操作,
    
因此void指针常用在:

+ 内存操作
+ 定义函数时,如果函数的参数可以是任意类型指针，那么应声明其参数为`void*`
+ 作为指向函数的指针


## *深入挖掘存储器

要真正理解什么是指针,我们需要了解一下计算机中内存是如何被使用的.这部分其实对大多数python程序员,甚至对多数程序员都是超纲内容,编程语言产生的目的就是让开发者可以不用关心计算机的内部实现,专门学这个就本末倒置了.但这对于一些特殊情况却是很有用的,比如在资源相当有限的单片机上时,我们操作资源的颗粒度就越小越好,而语言的抽象层级越低越好.这也就是为啥嵌入式一般都是用C语言来写.

一般来说系统会在一个进程被创建后为其开辟一段内存空间,而针对于C/C++,这段空间的结构是这样:




名称|内容
---|---
代码段	|可执行代码、字符串常量
数据段 |已初始化全局变量、已初始化全局静态变量、局部静态变量、常量数据
BSS段	|未初始化全局变量，未初始化全局静态变量
栈	|局部变量、函数参数
堆| 动态内存分配


而如何找到这些段中对应的内容呢,这就需要记录存储器单元的地址.比如如下这段代码:

```Cpp

int y = 1;

int main(){
    int x = 4;
    return 0;
}

```

y就存放在数据段,x就存放在栈中,假设y的存储器地址为100000,那就是说1000000地址的存储器其中存放的是int类型的一个值为1的内容.

而这个地址的编号也就是使用取地址符`&`取出来的东西了.而所谓的指针也就是存放这个地址的空间了.

而我们需要知道这段地址中的内容是什么类型,这样程序才知道要如何处理,所以指针也需要类型.

## 为什么要用指针?

如上面所说,指针是保存的一个变量的内存地址,那为什么要这么做呢?

### C语言按值传递参数


和python中不同,C语言调用函数的方式是严格按值传递,

```Cpp

int y = 1;

int add(int x,int y){
    return x+y
}

int main(){
    int x = 4;
    int z = add(x,y);
    return 0;
}

```

上例中的add函数接收两个int类型的参数,在被调用时,传入的参数值会按顺序被复制给函数对应的栈中对应的变量中.

这样的好处是函数的结果的不会因为外部数据变化而受到影响,坏处就是也影响不到外面的变量数据.而如果希望函数改变外部数据,那就应该使用指针.因为传入的是一段地址,所以只要根据这段地址找到这个变量的值就行.

# 引用(reference)

引用就是某一变量(目标)的一个别名,对引用的操作与对变量直接操作完全一样.

引用的声明方法:

```C++
类型标识符 &引用名=目标变量名；
```
如下,定义引用ra,它是变量a的引用,即别名:

```C++
int a;

int &ra=a;
```

1. &在此不是求地址运算符,而是起标识作用.

2. 类型标识符是指目标变量的类型.

3. 声明引用时,必须同时对其进行初始化.

4. 引用声明完毕后,相当于目标变量有两个名称即该目标原名称和引用名,且不能再把该引用名作为其他变量名的别名.

5. 声明一个引用,不是新定义了一个变量,它只表示该引用名是目标变量名的一个别名,它本身不是一种数据类型,因此引用本身不占存储单元,系统也不给引用分配存储单元.故:对引用求地址,就是对目标变量求地址.&ra与&a相等.

6. 不能建立数组的引用.因为数组是一个由若干个元素所组成的集合,所以无法建立一个数组的别名。


In [14]:
%%writefile ../src/C1/section5/reference_test.cpp
#include<iostream>
using std::cout;
using std::endl;
        
int main(){
    int a=5;
    int &b=a;
    b=6;
    cout<<"a="<<a<<",b="<<b<<endl;//a=6,b=6
    int c=7;
    b=c;
    cout<<"a="<<a<<",b="<<b<<endl;//a=7,b=7
    return 0;
}

Writing ../src/C1/section5/reference_test.cpp


In [15]:
!g++-7 -o ../bin/reference_test ../src/C1/section5/reference_test.cpp

In [16]:
!../bin/reference_test

a=6,b=6
a=7,b=7
